In [1]:
import re

In [2]:
# http요청 응답, html 구문분석 패키지
import requests
from bs4 import BeautifulSoup

In [3]:
# 셀레니움 브라우저의 동작을 자동화 하는 패키지
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [4]:
# 데이터 분석 패키지
import pandas as pd

In [5]:
# etf_name = input('ETF상품명을 입력하세요')
# etf_name = 'TIGER 2차전지테마'
etf_name = '에셋플러스 글로벌플랫폼액티브'

In [6]:
url = 'https://finance.naver.com/sise/etf.naver' # ETF 전체 목록 페이지
res = requests.get(url)

In [7]:
res.status_code

200

In [8]:
# 동적으로 요소가 생성되므로 찾고자 하는 정보를 찾을 수 없음
pattern = re.compile(r'2차전지')
pattern.findall(res.text)

[]

In [9]:
# 브라우저 종료 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option('detach', True)
# 웹드라이버를 이용한 브라우저 제어
driver = webdriver.Chrome(options=chrome_options)

#페이지가 모두 로드될때 까지 최대 1초 대기
driver.implicitly_wait(1)

In [10]:
driver.get(url)

In [11]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [12]:
# etf_name = 'KODEX 200'
# etf_name = 'TIGER 2차전지테마'

# etf_name = 'KODEX 단기채권PLUS'

In [13]:
detail_url = 'http://finance.naver.com'+soup.select_one('#etfItemTable').find('a',string=etf_name).attrs['href']

In [14]:
detail_url

'http://finance.naver.com/item/main.naver?code=407830'

In [15]:
driver.get(detail_url)

In [16]:
trs = driver.find_elements(By.CSS_SELECTOR, '#content > div.section.etf_asset > table > tbody > tr')
len(trs)

17

In [17]:
filtered_rows = []

for tr in trs[1:]:
    # tr내의 모든 td요소 찾기 
    tds = tr.find_elements(By.CSS_SELECTOR, 'td')

    # 'blank, division' 클래스가 포함된 td가 있는지 체크
    if all('blank' not in td.get_attribute('class') and 
           'division' not in td.get_attribute('class') for td in tds) :
        
        filtered_rows.append([td.text for td in tr.find_elements(By.CSS_SELECTOR,'td')])
        
print(len(filtered_rows))
print(filtered_rows)

10
[['XIAOMI CORP-CLASS B', '9,189', '-'], ['Coupang Inc', '860', '-'], ['ROBINHOOD MARKETS INC - A', '595', '-'], ['TESLA INC', '536', '-'], ['Palantir Technologies Inc', '484', '-'], ['HIMS & HERS HEALTH INC', '365', '-'], ['Shopify', '317', '-'], ['Amazon', '232', '-'], ['Alphabet Inc. Class C', '201', '-'], ['REDDIT INC', '200', '-']]


In [18]:
columns = [th.text for th in trs[0].find_elements(By.CSS_SELECTOR,'th')]
columns

['구성종목(구성자산)', '주식수(계약수)', '구성비중']

In [19]:
df = pd.DataFrame(filtered_rows,columns=columns)

In [20]:
df

,구성종목(구성자산),주식수(계약수),구성비중
0,XIAOMI CORP-CLASS B,"9,189",-
1,Coupang Inc,860,-
2,ROBINHOOD MARKETS INC - A,595,-
3,TESLA INC,536,-
4,Palantir Technologies Inc,484,-
5,HIMS & HERS HEALTH INC,365,-
6,Shopify,317,-
7,Amazon,232,-
8,Alphabet Inc. Class C,201,-
9,REDDIT INC,200,-


In [262]:
driver.quit()